In [1]:
import os
import re
import sys
import json
import time
import random
import requests
from hashlib import md5
from pyquery import PyQuery as pq
from urllib.request import urlretrieve

In [2]:
url_base = 'https://www.instagram.com/'
uri = 'https://www.instagram.com/graphql/query/?query_hash=a5164aed103f24b03e7b7747a2d94e3c&variables=%7B%22id%22%3A%22{user_id}%22%2C%22first%22%3A12%2C%22after%22%3A%22{cursor}%22%7D'


headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
    'cookie': 'mid=W9xfWQALAAG9F7peXFNb0mG8Y1I0; mcd=3; fbm_124024574287414=base_domain=.instagram.com; shbid=16651; shbts=1555339979.839692; rur=FTW; fbsr_124024574287414=Eog1bfmCpYxVmiVFk0KDUmW4YgYCM8sIa3e-KPil2lk.eyJjb2RlIjoiQVFENUdDTEs1YUNYc0hsUzIwS1JxeFVTc0JMcUl3bzdPRlR5S1JFMWNRQVZpNlBqZE0zeUEwcWhxWC1VOFZDYmFkeDV3QmppWmtBZjNDZlBWTUlJN1ZqNnhxQlpFTGpsbmVoNkJkYl9MSVJlX3l3dmJrRnBic3Y2QTk0SU9lU2JmM0V0WXdyTk1YdGptWHhNMEwxdno3S2tYNXZDWnEwWjFrLUVOUkgtRjNoWG5oYUNPQ2RMeFdMTXdiZ1p2ZHcyYzRJMW5CWG5kWDF3amdPbWpPWERyQW03RlJGai1aa0tnX0xIUGlKcnZldVljeDVjQzlIZm9PUDVNenBHV09BVjd5TnY4c3FJR1VkT2FlbS1PM1hXUVNSaWpFZzVIVGpRSlUxdF9aZFQ2VFJNQ3dqQXJUUHBHbDVMb1pLM0N6OWNkZl9hRXNnMGVmUWNfT0YyMklNTWltVkwiLCJ1c2VyX2lkIjoiMTAwMDAwNDQwMDQwNTM5IiwiYWxnb3JpdGhtIjoiSE1BQy1TSEEyNTYiLCJpc3N1ZWRfYXQiOjE1NTU0MzU0ODV9; csrftoken=pR4Wv3I3er1JHNpqMXxZ8ouyuUO8ns8K; ds_user_id=589591016; sessionid=589591016%3A6lQ8SgQPDFEdJ6%3A3; urlgen="{\"36.226.236.8\": 3462}:1hGRpA:gMlFSsadLkLQSc0evp6JLpfgv-M"'
}


In [9]:
def get_html(url):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            print('1请求网页源代码错误, 错误状态码：', response.status_code)
    except Exception as e:
        print(e)
        return None


def get_json(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            print('2请求网页json错误, 错误状态码：', response.status_code)
    except Exception as e:
        print(e)
        time.sleep(60 + float(random.randint(1, 4000))/100)
        return get_json(url)


def get_content(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            return response.content
        else:
            print('3请求照片二进制流错误, 错误状态码：', response.status_code)
    except Exception as e:
        print(e)
        return None


def get_urls(html):
    num = 1
    urls = []
    user_id = re.findall('"profilePage_([0-9]+)"', html, re.S)[0]
    print('user_id：' + user_id)
    doc = pq(html)
    items = doc('script[type="text/javascript"]').items()
    for item in items:
        if item.text().strip().startswith('window._sharedData'):
            js_data = json.loads(item.text()[21:-1], encoding='utf-8')
            edges = js_data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"]
            page_info = js_data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]['page_info']
            cursor = page_info['end_cursor']
            flag = page_info['has_next_page']
            for edge in edges:
                if edge['node']['display_url']:
                    display_url = edge['node']['display_url']
                    print(display_url)
                    #urlretrieve(display_url, './img/image'+str(num)+'.png')
                    num+=1
                    urls.append(display_url)
            print(cursor, flag)
    while flag:
        url = uri.format(user_id=user_id, cursor=cursor)
        js_data= get_json(url)
        print( url)
        infos = js_data['data']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        for info in infos:
            if info['node']['is_video']:
                video_url = info['node']['video_url']
                if video_url:
                    print(video_url)
                    urls.append(video_url)
            else:
                if info['node']['display_url']:
                    display_url = info['node']['display_url']
                    print(display_url)
                    #urlretrieve(display_url, './img/image'+str(num)+'.png')
                    num+=1
                    urls.append(display_url)
        print(cursor, flag)
        # time.sleep(4 + float(random.randint(1, 800))/200)    # if count > 2000, turn on
    return urls


def main(user):
    url = url_base + user + '/'
    print(url)
    html = get_html(url)
    urls = get_urls(html)
    dirpath = r'C:\Users\Rjunjun\python-ml\Instagram\{0}'.format(user)
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)
    for i in range(len(urls)):
        print('\n正在下载第{0}张： '.format(i) + urls[i], ' 还剩{0}张'.format(len(urls)-i-1))
        try:
            urlretrieve(urls[i], './'+user_name+'/image'+str(i)+'.png')
           # content = get_content(urls[i])
            #file_path = r'C:\Users\Rjunjun\python-ml\Instagram\{0}\{1}.{2}'.format(user, md5(content).hexdigest(), urls[i][-43:-40])
          
        except Exception as e:
            print(e)
            print('这张图片or视频下载失败')





In [12]:
if __name__ == '__main__':
    user_name = "katoarmy2.0"
    os.makedirs('./'+user_name+'/', exist_ok=True)
    start = time.time()
    main(user_name)
    print('Complete!!!!!!!!!!')
    end = time.time()
    spend = end - start
    hour = spend // 3600
    minu = (spend - 3600 * hour) // 60
    sec = spend - 3600 * hour - 60 * minu
    print(f'一共花费了{hour}小时{minu}分钟{sec}秒')

https://www.instagram.com/katoarmy2.0/
user_id：8194964983
https://instagram.ftpe8-3.fna.fbcdn.net/vp/dedf414b31807dcfeeb1aeb7f18417dd/5D3F7C17/t51.2885-15/e35/56618683_130004144820918_5461693490427640241_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/981e769b138ff2be05caafac21fceba3/5D43205F/t51.2885-15/e35/57558563_2088386484592540_7944497428283494180_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/dac9b14af221ec658930810557e6bf7d/5D3E686D/t51.2885-15/e35/57253579_592075917962538_1002770808836358708_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/b8362647fc1e7fd29c06e7ccbc931ebf/5D329BDE/t51.2885-15/e35/56531400_438748066898902_1183613484539329217_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/52442cce3b94534551e1f956e2741ad1/5D3B1B7F/t51.2885-15/e35/57506311_135245847609168_8082937340036269921_n.jpg?_nc_ht=instagram.ftpe8-3.

https://www.instagram.com/graphql/query/?query_hash=a5164aed103f24b03e7b7747a2d94e3c&variables=%7B%22id%22%3A%228194964983%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFCeVpRSnZVbDVjQkN6bzExd2dSd2ZBU3ZnMHZINlE5QWxHdkdidmZiM3hVZl9jQ3ZkWnBJNVZ3ZXJuNEoxQlFadnRWYXpqZXRjOUNOWmtqRkNFT0dKTw==%22%7D
https://instagram.ftpe8-3.fna.fbcdn.net/vp/68502bb1a5e57a6b6c19180b4fbec3ef/5D3A5A0F/t51.2885-15/e35/54513724_133154617753186_1323735507111583396_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/4eec7f8a8ddaf2b389ea7762b7a76e1a/5D3B396A/t51.2885-15/e35/54511431_637615243367580_4491060052375405564_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/dcae33ce90e67f5a7810595fe8fe0636/5D76F132/t51.2885-15/e35/56464458_418732798887816_2497522231340370304_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/9ffb4bbb9d585597a558042c2a8c6dff/5D3A11EB/t51.2885-15/e35/54800687_365840790693614_4306443776

https://www.instagram.com/graphql/query/?query_hash=a5164aed103f24b03e7b7747a2d94e3c&variables=%7B%22id%22%3A%228194964983%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFBTUtsZnNxYk1HR2NfR0RoWlkwVkNqQXh0VVpocndmVVUwZVJxTGtpN2ZobGFhTGxEX3JQZ0tBNExmY2paRnVYNTdyMEYwZW1zVDgxLTE0NHFaYzNReA==%22%7D
https://instagram.ftpe8-3.fna.fbcdn.net/vp/79dccaf753a7b186a4a8cbff528e0a67/5D538A6D/t51.2885-15/e35/54196530_123318648822483_7841786287295519261_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/b6e322414d113d2e93aa18e829047f0b/5D40EE89/t51.2885-15/e35/54732000_319861655340861_3340764950447954728_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/d1c4fe400bb5f9ac0d678bb11a8b77a7/5D393706/t51.2885-15/e35/54512789_2267542436852921_560851299882311469_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/05884a07bfe195e57dce8f5f0004c971/5D43768D/t51.2885-15/e35/54512752_2197452433705378_336160360

https://www.instagram.com/graphql/query/?query_hash=a5164aed103f24b03e7b7747a2d94e3c&variables=%7B%22id%22%3A%228194964983%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFEa2pWNkZTQzVSeVNxMGFYNE0yRl9vVm5ZVTlkRm52V1Q1UXVRck9PeVo2QmZhWHFOenVyMk5Lb2dseURwWjRnMGgwajhQLUU5ZVR3c0NxRUpUb2lSNQ==%22%7D
https://instagram.ftpe8-3.fna.fbcdn.net/vp/f9f226c21868ee381cd5f02eb51c50a0/5D783412/t51.2885-15/e35/53030758_2162276067182958_8888881218441147934_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/6a820cb2a21f8b86ccb70bfdfab0b9e4/5D31444A/t51.2885-15/e35/54800859_369462440312433_3993474471080727072_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/9fbf8ab82e7fa88d548b61e6d5a8e22a/5D778177/t51.2885-15/e35/53155881_324858864745260_4404072091983417250_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/1ce566756cb90eba3815a7ea025add63/5D50B3C3/t51.2885-15/e35/53544447_2299854393404251_84686893

https://www.instagram.com/graphql/query/?query_hash=a5164aed103f24b03e7b7747a2d94e3c&variables=%7B%22id%22%3A%228194964983%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFDUVNWZlRJU3Y3dmtIRjBLQkR6d0NyZXhDZkkyZEtGNTF0NmVjZ1E3aDJ3dkQtRU5tR1FtNmE4Nk9GTWs4TWNMcm5LMFdnUEhIcXNldE1uWWNKZDhnRg==%22%7D
https://instagram.ftpe8-3.fna.fbcdn.net/vp/9dfc32f025e8bfe5475ccd084b8369ad/5D3F2E3C/t51.2885-15/e35/53909928_155649008710647_7680259113242450245_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/d30d5b008a3479fc42d43618ef009a05/5D3099A6/t51.2885-15/e35/53034692_144063993300425_5678878410648261776_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/3fb79e0d012db73b861ee8367d0f831f/5D33B8B2/t51.2885-15/e35/52940788_333432130620539_4839363503248676715_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net
https://instagram.ftpe8-3.fna.fbcdn.net/vp/69681c84caa7fed9e6d550551b4beee4/5D2EEC97/t51.2885-15/e35/53026516_270578233868773_7454805387

...


正在下载第1253张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/27b35e2b9c3a4b608da247642a4b945e/5D4AEC0E/t51.2885-15/e35/37906451_418202255405992_5494189205187723264_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩180张

正在下载第1254张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/a83f6c4f2de202d421aac52c1ca435d8/5D3A110B/t51.2885-15/e35/38035115_403893396681729_4505382322266177536_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩179张

正在下载第1255张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/189fc7f74b72b24446d1af5e9bdf73f6/5D4A8728/t51.2885-15/e35/37878291_318274012075161_2773182794442997760_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩178张

正在下载第1256张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/f9ec3e24b6904bc2e47373bc92331b0a/5D3E3AA3/t51.2885-15/e35/37984017_288819921925506_4418752398114160640_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩177张

正在下载第1257张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/7cd6c53b9fcfd8296cc9951e9fd7ea8e/5D45CFAA/t51.2885-15/e35/38194810_1694868667292683_647969394115


正在下载第1294张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/d6f49f6b83ed4751753dabe1c038c510/5D3CC03F/t51.2885-15/e35/37658838_212352282783074_7980769660897329152_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩139张

正在下载第1295张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/85bf49baa04c653cebdd259e39047725/5D35440B/t51.2885-15/e35/37708349_2482446788648238_3013577452644466688_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩138张

正在下载第1296张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/72f1795608fd0c41eece46766484cd02/5D31809C/t51.2885-15/e35/37600631_2097707687155787_7933209138681610240_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩137张

正在下载第1297张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/83be4f37dbdd7807157a4f96fd233297/5D2F923E/t51.2885-15/e35/36926705_257838135031502_6028538971384971264_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net  还剩136张

正在下载第1298张： https://instagram.ftpe8-3.fna.fbcdn.net/vp/ad9c45468e1e4c558374bd61d7532429/5D2D04CF/t51.2885-15/e35/37143818_1761786733870230_1879032687


正在下载第1335张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/37e8b1c67724b32e9cc560e986ef965e/5D43BE15/t51.2885-15/e35/37086096_676623772686428_2440770257221058560_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩98张

正在下载第1336张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/4d5c01e7e541c435bce68750bf8399f9/5D77D324/t51.2885-15/e35/37284143_718987441779079_8199701199289581568_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩97张

正在下载第1337张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/3932faf864e555b95e3c0630a3f199bc/5D5438A5/t51.2885-15/e35/37508130_695288124142674_1276812250031587328_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩96张

正在下载第1338张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/e77d00bcc04e13df1f53228fd8225dca/5D53398A/t51.2885-15/e35/37056114_225917727942379_5311158190321172480_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩95张

正在下载第1339张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/f3feb7ce78b3351ecbd861303568679f/5D4442F4/t51.2885-15/e35/37134197_279664189449008_89204661568845905


正在下载第1375张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/ed5e77a1f884b24f82ec24c9ffc46c95/5D529F42/t51.2885-15/e35/36891136_2003971506331664_7160635287675076608_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩58张

正在下载第1376张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/8840f5d8f164c20bb98701e49a9737a9/5D3ABDCE/t51.2885-15/e35/36869654_176903396514726_7906088971417944064_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩57张

正在下载第1377张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/370dd5c2e88bc34f540ed9685e58331a/5D3F4249/t51.2885-15/e35/36775200_404808896593521_584547660325715968_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩56张

正在下载第1378张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/0e81a7df4b5ac01272f237d7b88164af/5D3B82E1/t51.2885-15/e35/36881681_1735838626470556_8055560782575828992_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩55张

正在下载第1379张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/b220d5a6432956be317821e0bd118948/5D4837BF/t51.2885-15/e35/36875395_442221842928892_8997963022060224


正在下载第1417张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/ba7d0510ad591f5e5cae02ee6fe365a7/5D3BEEED/t51.2885-15/e35/37063326_2063985980532022_6622979766754476032_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩16张

正在下载第1418张： https://scontent.cdninstagram.com/vp/65764eb62276b5ef2893ccf37d53cd0a/5CB8CA3A/t50.2886-16/36650155_1999916660040153_5095748861699551258_n.mp4?_nc_ht=scontent.cdninstagram.com  还剩15张

正在下载第1419张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/1b68e4ab5f6af94908046bed78d5831c/5D3EE66C/t51.2885-15/e35/36868608_376513772877903_3252508733883809792_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩14张

正在下载第1420张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/eeb5fa3b2987b0a57f29cec60c5fece7/5D39E364/t51.2885-15/e35/36656021_850329648499652_1173596051240648704_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net  还剩13张

正在下载第1421张： https://instagram.ftpe8-2.fna.fbcdn.net/vp/237f665df8bf5747007152d92f26772b/5D75D80C/t51.2885-15/e35/36552112_209970323049846_5830971510856089600_n.jpg?_nc_h